# Testing Usage of Adapter Adversarial Reweighting

This notebooks is based off of: https://github.com/Trusted-AI/AIF360/blob/master/examples/demo_adversarial_debiasing.ipynb

## Imports & Configs

In [2]:
%matplotlib inline
# Load all necessary packages
import sys
sys.path.append("../")
from aif360.datasets import BinaryLabelDataset
from aif360.datasets import AdultDataset, GermanDataset, CompasDataset
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.metrics import ClassificationMetric
from aif360.metrics.utils import compute_boolean_conditioning_vector

from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions import load_preproc_data_adult, load_preproc_data_compas, load_preproc_data_german

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, MaxAbsScaler
from sklearn.metrics import accuracy_score

from IPython.display import Markdown, display
import matplotlib.pyplot as plt

import tensorflow.compat.v1 as tf
tf.disable_eager_execution()

In [3]:
from aif360.algorithms.inprocessing.adversarial_debiasing import AdversarialDebiasing

In [4]:
from adversarial_reweighting_adapter import AdversarialReweightingAdapter

## Loading Dataset

In [5]:
# Get the dataset and split into train and test
dataset_orig = load_preproc_data_adult()

privileged_groups = [{'sex': 1}]
unprivileged_groups = [{'sex': 0}]

dataset_orig_train, dataset_orig_test = dataset_orig.split([0.7], shuffle=True)

In [6]:
# print out some labels, names, etc.
display(Markdown("#### Training Dataset shape"))
print(dataset_orig_train.features.shape)
display(Markdown("#### Favorable and unfavorable labels"))
print(dataset_orig_train.favorable_label, dataset_orig_train.unfavorable_label)
display(Markdown("#### Protected attribute names"))
print(dataset_orig_train.protected_attribute_names)
display(Markdown("#### Privileged and unprivileged protected attribute values"))
print(dataset_orig_train.privileged_protected_attributes, 
      dataset_orig_train.unprivileged_protected_attributes)
display(Markdown("#### Dataset feature names"))
print(dataset_orig_train.feature_names)

#### Training Dataset shape

(34189, 18)


#### Favorable and unfavorable labels

1.0 0.0


#### Protected attribute names

['sex', 'race']


#### Privileged and unprivileged protected attribute values

[array([1.]), array([1.])] [array([0.]), array([0.])]


#### Dataset feature names

['race', 'sex', 'Age (decade)=10', 'Age (decade)=20', 'Age (decade)=30', 'Age (decade)=40', 'Age (decade)=50', 'Age (decade)=60', 'Age (decade)=>=70', 'Education Years=6', 'Education Years=7', 'Education Years=8', 'Education Years=9', 'Education Years=10', 'Education Years=11', 'Education Years=12', 'Education Years=<6', 'Education Years=>12']


In [7]:
# Metric for the original dataset
metric_orig_train = BinaryLabelDatasetMetric(dataset_orig_train, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
display(Markdown("#### Original training dataset"))
print("Train set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_orig_train.mean_difference())
metric_orig_test = BinaryLabelDatasetMetric(dataset_orig_test, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
print("Test set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_orig_test.mean_difference())


#### Original training dataset

Train set: Difference in mean outcomes between unprivileged and privileged groups = -0.189703
Test set: Difference in mean outcomes between unprivileged and privileged groups = -0.205752


In [8]:
# min_max_scaler = MaxAbsScaler()
# dataset_orig_train.features = min_max_scaler.fit_transform(dataset_orig_train.features)
# dataset_orig_test.features = min_max_scaler.transform(dataset_orig_test.features)
# metric_scaled_train = BinaryLabelDatasetMetric(dataset_orig_train, 
#                              unprivileged_groups=unprivileged_groups,
#                              privileged_groups=privileged_groups)
# display(Markdown("#### Scaled dataset - Verify that the scaling does not affect the group label statistics"))
# print("Train set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_scaled_train.mean_difference())
# metric_scaled_test = BinaryLabelDatasetMetric(dataset_orig_test, 
#                              unprivileged_groups=unprivileged_groups,
#                              privileged_groups=privileged_groups)
# print("Test set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_scaled_test.mean_difference())

## Baseline model

In [9]:
# Load post-processing algorithm that equalizes the odds
# Learn parameters with debias set to False
sess = tf.Session()
plain_model = AdversarialDebiasing(
    privileged_groups = privileged_groups,
    unprivileged_groups = unprivileged_groups,
    scope_name='plain_classifier',
    debias=False,
    sess=sess
)

2022-04-06 19:08:14.353095: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2022-04-06 19:08:14.400927: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f9d2797a620 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2022-04-06 19:08:14.400950: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version


In [10]:
plain_model.fit(dataset_orig_train)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
epoch 0; iter: 0; batch classifier loss: 0.661927
epoch 0; iter: 200; batch classifier loss: 0.401323
epoch 1; iter: 0; batch classifier loss: 0.457101
epoch 1; iter: 200; batch classifier loss: 0.441656
epoch 2; iter: 0; batch classifier loss: 0.399996
epoch 2; iter: 200; batch classifier loss: 0.395545
epoch 3; iter: 0; batch classifier loss: 0.470467
epoch 3; iter: 200; batch classifier loss: 0.418490
epoch 4; iter: 0; batch classifier loss: 0.461544
epoch 4; iter: 200; batch classifier loss: 0.466973
epoch 5; iter: 0; batch classifier loss: 0.405223
epoch 5; iter: 200; batch classifier loss: 0.374264
epoch 6; iter: 0; batch classifier loss: 0.488470
epoch 6; iter: 200; batch classifier loss: 0.333958
epoch 7; iter: 0; batch classifier loss: 0.456750
epoch 7; iter: 200; batch c

In [11]:
# Apply the plain model to test data
dataset_nodebiasing_train = plain_model.predict(dataset_orig_train)
dataset_nodebiasing_test = plain_model.predict(dataset_orig_test)

In [12]:
# Metrics for the dataset from plain model (without debiasing)
display(Markdown("#### Plain model - without debiasing - dataset metrics"))
metric_dataset_nodebiasing_train = BinaryLabelDatasetMetric(dataset_nodebiasing_train, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)

print("Train set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_dataset_nodebiasing_train.mean_difference())

metric_dataset_nodebiasing_test = BinaryLabelDatasetMetric(dataset_nodebiasing_test, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)

print("Test set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_dataset_nodebiasing_test.mean_difference())

display(Markdown("#### Plain model - without debiasing - classification metrics"))
classified_metric_nodebiasing_test = ClassificationMetric(dataset_orig_test, 
                                                 dataset_nodebiasing_test,
                                                 unprivileged_groups=unprivileged_groups,
                                                 privileged_groups=privileged_groups)
print("Test set: Classification accuracy = %f" % classified_metric_nodebiasing_test.accuracy())
TPR = classified_metric_nodebiasing_test.true_positive_rate()
TNR = classified_metric_nodebiasing_test.true_negative_rate()
bal_acc_nodebiasing_test = 0.5*(TPR+TNR)
print("Test set: Balanced classification accuracy = %f" % bal_acc_nodebiasing_test)
print("Test set: Disparate impact = %f" % classified_metric_nodebiasing_test.disparate_impact())
print("Test set: Equal opportunity difference = %f" % classified_metric_nodebiasing_test.equal_opportunity_difference())
print("Test set: Average odds difference = %f" % classified_metric_nodebiasing_test.average_odds_difference())
print("Test set: Theil_index = %f" % classified_metric_nodebiasing_test.theil_index())

#### Plain model - without debiasing - dataset metrics

Train set: Difference in mean outcomes between unprivileged and privileged groups = -0.216255
Test set: Difference in mean outcomes between unprivileged and privileged groups = -0.221813


#### Plain model - without debiasing - classification metrics

Test set: Classification accuracy = 0.802225
Test set: Balanced classification accuracy = 0.664768
Test set: Disparate impact = 0.000000
Test set: Equal opportunity difference = -0.464849
Test set: Average odds difference = -0.288294
Test set: Theil_index = 0.178652


In [13]:
sess.close()
tf.reset_default_graph()

## Adversarial Reweighting Model

In [14]:
from group_agnostic_fairness.data_utils.uci_adult_input import UCIAdultInput

In [15]:
FLAGS_dataset_base_dir = './group_agnostic_fairness/data/uci_adult'
FLAGS_train_file = f'{FLAGS_dataset_base_dir}/adult.data'
FLAGS_test_file = f'{FLAGS_dataset_base_dir}/adult.test'

In [16]:
adult_dataset_input = UCIAdultInput(
    dataset_base_dir=FLAGS_dataset_base_dir,
    train_file=FLAGS_train_file,
    test_file=FLAGS_test_file
)

In [17]:
sess = tf.Session()

In [18]:
model_output_dir = "./adv_reweighted_run_model"

In [19]:
reweighted_model = AdversarialReweightingAdapter(
    adult_dataset_input.feature_names,
    adult_dataset_input.target_column_name,
    model_output_dir,
    adult_dataset_input
)

UnparsedFlagAccessError: ignored